# CIFAR10

Train a simple deep CNN on the CIFAR10 small images dataset.

GPU run command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10_cnn.py

It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
Note: the data was pickled with Python 2, and some encoding issues might prevent you
from loading it in Python 3. You might have to load it in Python 2,
save it in a different format, load it in Python 3 and repickle it.

In [1]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

In [2]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

Using Theano backend.
Using gpu device 0: GeForce GTX 660 (CNMeM is disabled, cuDNN 5005)


In [7]:
batch_size = 32
nb_classes = 10
nb_epoch = 50

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

In [8]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [9]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [10]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [11]:
%%time 

data_augmentation = False

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True, verbose=2)
    
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
31s - loss: 1.7239 - acc: 0.3637 - val_loss: 1.3372 - val_acc: 0.5230
Epoch 2/50
31s - loss: 1.3256 - acc: 0.5218 - val_loss: 1.1659 - val_acc: 0.5830
Epoch 3/50
31s - loss: 1.1679 - acc: 0.5819 - val_loss: 1.0149 - val_acc: 0.6444
Epoch 4/50
31s - loss: 1.0545 - acc: 0.6263 - val_loss: 0.9089 - val_acc: 0.6824
Epoch 5/50
31s - loss: 0.9680 - acc: 0.6575 - val_loss: 0.8671 - val_acc: 0.6947
Epoch 6/50
31s - loss: 0.9050 - acc: 0.6813 - val_loss: 0.8470 - val_acc: 0.7075
Epoch 7/50
31s - loss: 0.8585 - acc: 0.6999 - val_loss: 0.7710 - val_acc: 0.7371
Epoch 8/50
31s - loss: 0.8145 - acc: 0.7157 - val_loss: 0.7717 - val_acc: 0.7336
Epoch 9/50
31s - loss: 0.7919 - acc: 0.7242 - val_loss: 0.7943 - val_acc: 0.7285
Epoch 10/50
31s - loss: 0.7645 - acc: 0.7332 - val_loss: 0.7762 - val_acc: 0.7377
Epoch 11/50
31s - loss: 0.7409 - acc: 0.7415 - val_loss: 0.7441 - val_acc: 0.7472
Epoch 12/50
31s - loss: 0.72

In [12]:
score = model.evaluate(X_test, Y_test, verbose=1)

 9984/10000 [============================>.] - ETA: 0s

In [15]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.734846554756
Test accuracy: 0.7602
